## ANALISIS DE DATOS DE FILMAFFINITY

In [1]:
import pandas as pd
import sqlite3

# ruta del csv
ruta_csv = "./data/filmaffinity_dataset.csv"

# Cargar el csv en un dataframe
df = pd.read_csv(ruta_csv)

# Conexión SQLite
conexion = sqlite3.connect("filmaffinity.db")

# Guardar el DataFrame como tabla en SQLite
df.to_sql("filmaffinity_tb", conexion, if_exists="replace", index=False)

print("Base de datos SQLite creada y datos importados")

cursor = conexion.cursor()

Base de datos SQLite creada y datos importados


In [4]:
consulta = """
           SELECT *
           FROM filmaffinity_tb
           LIMIT 5;
           """
resultado = pd.read_sql_query(consulta, conexion)
resultado

,index,Unnamed: 0,Título,Año,País,Dirección,Reparto,Nota,Tipo filme,Género
0,0,0,'49-'17,1917,Estados Unidos,Ruth Ann Baldwin,"Joseph W. Girard, Leo Pierson, William Dyer, M...",--,Película,Western
1,1,1,"10,000 Years B.C. (C)",1916,Estados Unidos,Willis H. O'Brien,None,"5,1",Cortometraje,Comedia
2,2,2,1812,1912,Rusia,"Vasili Goncharov, Kai Hansen, Aleksandr Uralsky","Pavel Knorr, Vasili Goncharov, Aleksandra Gonc...",--,Película,Drama
3,3,3,20.000 leguas de viaje submarino (C),1907,Francia,Georges Méliès,Georges Méliès,"6,0",Cortometraje,Fantástico
4,4,4,A Bad Case (C),1909,Francia,Émile Cohl,None,"5,3",Cortometraje,Comedia


# 1. Limpieza de data
En esta face haremos lo siguiente:

 1. Hacer un copia de la base de datos original para luego poder trabajar en esta copia y no afectar a la original.
 2. Luego pasamos a hacer la limpieza que consiste en:
      
      ✅ Datos faltantes (Nulos).

      ✅ Registros duplicados.

      ✅ Formatos inconsistentes (fechas, nombres, números).
      
      ✅ Valores atípicos (outliers).

### Hacer una copia de la db original para trabajar con esta copia

In [ ]:
# Cerrar conexión con
# conexion.close()